# 1. 데이터 합치기

### Pandas 객체에 저장된 데이터는 여러 내장함수를 통해 합칠 수 있다.
* pandas.merge : 하나이상의 키를 기준으로 df의 row를 합친다. (SQL의 join)
* pandas.concat : 하나의 축을 따라 객체를 이어 붙인다.
* pandas.combine_first : 두 객체를 포개서 한 객체에 누락된 데이터를 다른객체에 있는 값으로 채울 수 있도록 한다.

## 1.1 데이터베이스 스타일로 DataFrame 합치기

In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [3]:
df1 = DataFrame({'key': ['b','b','a','c','a','a','b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a','b','d'],
                 'data2': range(3)})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [6]:
# merge : 키를 지정하지 않으면 겹치는 칼럼의 이름을 키로 사용
pd.merge(df1,df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [7]:
# 키는 항상 지정하는 습관을 들이는게 좋다.
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
# 겹치는 칼럼이 없으면 따로 지정하면 된다. (left_on, right_on) -> 나름 활용할 부분이 많을 꺼 같음
df3 = DataFrame({'lkey': ['b','b','a','c','a','a','b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a','b','d'],
                 'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


### merge 함수의 기본 : 내부조인(inner join)
* how-option
- default : inner (교집합)
- left, right : 포함된 로우 기준 전부 포함
- outer : 합집합 반환

In [9]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [10]:
# 다 대 다 병합 -> 두 로우의 데카르트 곱 (직관적이지 않다.)
df1 = DataFrame({'key': ['b','b','a','c','a','b'],
                 'data1': range(6)})
df2 = DataFrame({'key': ['a','b','a','b','d'],
                 'data2': range(5)})

pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [11]:
pd.merge(df1, df2, on='key', how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [12]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'lval': [4, 5, 6, 7]})

In [13]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [14]:
right

,key1,key2,lval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [15]:
# 여러개의 키를 병합 -> 칼럼 이름의 list를 넘기기
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [16]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval_x,key2_y,lval_y
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [17]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### merge 함수 인자 목록
- key : column name / 
* left : merge하려는 DF 중 왼쪽에 위치한 DF
* right : merge하려는 DF 중 오른쪽에 위치한 DF
* how : 조인 방법. inner(default) / left / right / outer 
* on : 조인하려는 row 이름(=column name=join_key). 반드시 두 DF에 모두 있는 이름이어야 함. 명지되지 않을 경우 자동으로 동시에 있는 이름으로 설정
* left_on : 조인키로 사용할 left DataFrame의 칼럼
* right_on : 조인키로 사용할 rigth DataFrame의 칼럼
* left_index : 조인키로 사용할 left DataFrame의 index_row(다중 색인일 경우 키)
* right_index : 조인키로 사용할 right DataFrame의 index_row(다중 색인일 경우 키)
* sort : 조인키에 따라 병합된 데이터를 사전순으로 정렬. default = True
* suffixes : 칼럼 이름이 겹칠경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플. default = ('_x','_y')